In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.core.display import display, HTML

In [2]:
os.environ["CUDA_VISIBLE_DIVICES"] = "9"
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)
display(HTML("<style>.container { width:98% !important;}</style>"))

In [3]:
imagenet_label_path = "/raid/peterchen/datasets/imagenet/2012/imagenet_class_index.json"

In [5]:
with open(imagenet_label_path) as f:
    imagenet_label = json.load(f)

In [65]:
df

,0,1
0,n01440764,tench
1,n01443537,goldfish
2,n01484850,great_white_shark
3,n01491361,tiger_shark
4,n01494475,hammerhead
...,...,...
995,n13044778,earthstar
996,n13052670,hen-of-the-woods
997,n13054560,bolete
998,n13133613,ear
